Tesina de bachelor: filologia hispánica

# Collect filenames
This part is concerned with collecting all the filenames of the relevant files

In [1]:
import os
import fnmatch
import copy
import unittest
import csv

In [2]:
import es_core_news_sm
nlp = es_core_news_sm.load()

/home/nienke/Documents/jupyternotebookvenv/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
def find_files(root_folder='../BVC'):
    """
    Find all files in the BVC folder 
    Code from stack overflow: https://stackoverflow.com/questions/2186525/how-to-use-glob-to-find-files-recursively
    
    root_folder: the root folder for the files
    return: list of filenames in the root folder
    """
    
    matches = []
    for root, dirnames, filenames in os.walk(root_folder):
        for filename in fnmatch.filter(filenames, '*.xml'):
            matches.append(os.path.join(root, filename))
    return matches

In [4]:
print(find_files())
files = find_files()
nr_file_to_parse = 1010

['../BVC/8784.xml', '../BVC/449.xml', '../BVC/461.xml', '../BVC/164.xml', '../BVC/458.xml', '../BVC/8700.xml', '../BVC/23.xml', '../BVC/8781.xml', '../BVC/test.xml', '../BVC/138.xml', '../BVC/457.xml', '../BVC/8778.xml', '../BVC/8648.xml', '../BVC/8677.xml', '../BVC/5412.xml', '../BVC/1047.xml', '../BVC/8787.xml', '../BVC/90.xml', '../BVC/8709.xml', '../BVC/8703.xml', '../BVC/103.xml', '../BVC/8786.xml', '../BVC/8675.xml', '../BVC/8455.xml', '../BVC/1313.xml', '../BVC/8108.xml', '../BVC/455.xml', '../BVC/8699.xml', '../BVC/10107.xml', '../BVC/8695.xml', '../BVC/8788.xml', '../BVC/8697.xml', '../BVC/8696.xml', '../BVC/8650.xml', '../BVC/8652.xml', '../BVC/6837.xml', '../BVC/8676.xml', '../BVC/8651.xml', '../BVC/8643.xml', '../BVC/8776.xml', '../BVC/1708.xml', '../BVC/8681.xml', '../BVC/8698.xml', '../BVC/8789.xml', '../BVC/8701.xml', '../BVC/453.xml', '../BVC/456.xml', '../BVC/184.xml', '../BVC/463.xml', '../BVC/462.xml', '../BVC/8678.xml', '../BVC/8692.xml', '../BVC/1011.xml', '../BVC/

# Parse xml file
This part is concerned with extracting the relevant information from the xml file

In [5]:
class MetaData:
    def __init__(self, year, title, author):
        '''
        year: the year in which the sentence was written
        text_title: title of the text
        original_text: the complete original text
        modern_text: the modern text with fixed spelling
        tagged_text: the nlp tagged text
        '''
        self.year = year
        self.text_title = title
        self.author = author
    
    def __repr__(self):
        return "%s by %s (%s)" % (self.text_title, self.author, self.year)
    
    def __cmp__(self, other):
        return cmp(self.year, other.year)

In [6]:
class Tag():
    def __init__(self, text, dep_, pos_, head):
        self.text = text
        self.dep_ = dep_
        self.pos_ = pos_
        self.head = head

In [7]:
# Word type sets
indefinite_articles = ['un', 'una', 'unos', 'unas']
definite_articles = ['el', 'la', 'los', 'las'] #vergeet 'al' niet!
pron_determiner = ['mi', 'tu', 'su', 'mis', 'tus', 'sus', 'nuestros', 'nuestras', 'nuestro', 'nuestra', 'vuestros', 'vuestras', 'vuestro', 'vuestra']

In [8]:
import enum
class Definiteness(enum.Enum):
    '''
    Codes the definiteness scale
    '''
    pronoun = 1
    name = 2
    definite = 3
    indef_spec = 4
    indef_nonspec = 5
    unknown = 6

class Order(enum.Enum):
    obj_first = 1
    both = 2
    verb_first = 3

In [9]:
class ObjectSentence():
    def __init__(self, metadata, subtree, orig_sentence):
        '''
        metadata: the metadata such as year of the sentence
        subtree: the nlp subtree object of the object sentence
        orig_sentence: the complete original sentence
        
        only pronoun objects? Check
        verb - Check (sort of, lemmatization would be nice)
        rel position verb and object
        animacy
        determinancy

        '''
        self.metadata = metadata
        self.tagged_text = []
        
        # Make dummy object if no subtree 
        if subtree is None:
            tagged_text = nlp(orig_sentence)
            for w in tagged_text:
                self.tagged_text.append(Tag(w.text, w.dep_, w.pos_, w.head))
            self.has_only_pron_objects()
            self.determine_definiteness()
            
        
        # Make normal object otherwise
        else:
            for w in subtree:
                self.tagged_text.append(Tag(w.text, w.dep_, w.pos_, w.head))

            self.has_a = orig_sentence.has_object_with_a
            self.original_text = orig_sentence.original_text
            self.modern_text = orig_sentence.modern_text

            self.has_only_pron_objects() 
            self.has_only_adj_objects()
            self.find_obj_verb()
            self.determine_definiteness()
            self.determine_order()
            #self.rel_pos_vo =    

            # Write code to locate the object
    
    
    def has_only_pron_objects(self):
        self.has_only_pron = True
        for w in self.tagged_text:
            if w.dep_ == 'obj' and w.pos_ != 'PRON':
                self.has_only_pron = False
    
    def has_only_adj_objects(self):
        self.has_only_adj = True
        for w in self.tagged_text:
            if w.dep_ == 'obj' and w.pos_ != 'ADJ':
                self.has_only_adj = False
    
    def find_obj_verb(self):
        for w in self.tagged_text:
            if w.dep_ == 'obj':
                self.verb = w.head
                return # Simplifying assumption: we assume the first verb is the one for the main obj sentence (if there are multiple verbs)
    
    def determine_order(self):
        order = ""
        for w in self.tagged_text:
            if w.dep_ == 'obj':
                order += "O"
            elif w.pos_ == "VERB":
                order += "V"
        self.order = order
    
    def determine_definiteness(self):
        # pseudo-code:
        # first check if the object is capitalized, then probably proper name and we will put it there
        # then look use_only_pron_objects, cause this already tells us it is the pronoun cat
        # then find determiners
        #     if det in definite_articles or in pron_determiner, then definite
        
        # edge case: uno de mis perros
        # heeft kennelijk een 'nmod' dependency, dus kan gebruikt worden mss om dit soort gevallen te vinden
        
        # We start with unknown and hope it gets overwritten by one of our cases
        self.determinacy = Definiteness.unknown
        
        # First find out if we have a case of "una/o de ...."
        for w in self.tagged_text:
            if w.dep_ == 'nmod':
                for v in self.tagged_text:
                    if v.text == str(w.head) and v.dep_ == 'obj':
                        self.determinacy = Definiteness.indef_spec
                        return
                    if str(v.head) == str(w.head) and v.dep_ == 'obj':
                        # we have found the object
                        self.determinacy = Definiteness.indef_spec
                        return
                    if str(w.head) == str(v.head) and v.dep_ == 'nummod': # for case 'veo dos de mis perros'
                        self.determinacy = Definiteness.indef_spec
                        return
                    
        # Second use the only pronoun objects function; if true, then we know our answer
        if self.has_only_pron:
            self.determinacy = Definiteness.pronoun
            return
        
        else:
            for w in self.tagged_text:
                if w.dep_ == 'obj':
                    # If proper name, it is a proper name, easy
                    if w.pos_ == 'PROPN':
                        self.determinacy = Definiteness.name
                        return
                    
                    # If it is a noun, we can start determining the determinacy of the determiner
                    elif w.pos_ == 'NOUN':
                        for v in self.tagged_text:
                            if str(v.head) == w.text:
                                
                                if v.text in definite_articles or v.text in pron_determiner:
                                    self.determinacy = Definiteness.definite
                                    return
                                elif v.text == 'al':
                                    self.determinacy = Definiteness.definite
                                    return
                                elif v.text in indefinite_articles:
                                    self.determinacy = Definiteness.indef_nonspec
                                    return
                                else:
                                    print(self, v.text, v.pos_)
                    
                    # We are not interested in the pronoun objects
                    elif w.pos_ == 'PRON': 
                        continue
                    else: # For debugging purposes, we want to know what the hell we encountered
                        print(self, w.text, w.pos_)
    
    def __cmp__(self, other):
        return cmp(self.metadata, other.metadata)
    
    def __repr__(self):
        s = "/"
        for w in self.tagged_text:
            s += w.text + " "
        return s + '/'

In [10]:
class MyTest(unittest.TestCase):
    def test_determine_def(self):
        '''
        TODO: make test cases of all sentences both with and without 'a'
        '''
        # Create dummy metadata
        metadata = MetaData("", "", "")
        
        #obj_sen = ObjectSentence(metadata, None, "Veo a mis hijos")
        
        
        self.assertEqual(Definiteness.name, ObjectSentence(metadata, None, "Veo a José").determinacy)
        self.assertEqual(Definiteness.name, ObjectSentence(metadata, None, "Mañana voy a ver RTVE").determinacy)
        
        self.assertEqual(Definiteness.pronoun, ObjectSentence(metadata, None, "No lo sé").determinacy)
        self.assertEqual(Definiteness.pronoun, ObjectSentence(metadata, None, "La veo a ella").determinacy)

        
        self.assertEqual(Definiteness.definite, ObjectSentence(metadata, None, "Veo a mis hijos").determinacy)
        self.assertEqual(Definiteness.definite, ObjectSentence(metadata, None, "Veo mis hijos").determinacy)
        self.assertEqual(Definiteness.definite, ObjectSentence(metadata, None, "Veo a la televisión").determinacy)
        self.assertEqual(Definiteness.definite, ObjectSentence(metadata, None, "Veo la tele").determinacy)
        self.assertEqual(Definiteness.definite, ObjectSentence(metadata, None, "Veo al doctor").determinacy)
        
        self.assertEqual(Definiteness.indef_spec, ObjectSentence(metadata, None, "Veo uno de mis hijos").determinacy)
        self.assertEqual(Definiteness.indef_spec, ObjectSentence(metadata, None, "Veo a uno de mis hijos").determinacy)
        self.assertEqual(Definiteness.indef_spec, ObjectSentence(metadata, None, "Veo dos de mis hijos").determinacy)
        self.assertEqual(Definiteness.indef_spec, ObjectSentence(metadata, None, "Veo a dos de mis hijos").determinacy)
        
        self.assertEqual(Definiteness.indef_nonspec, ObjectSentence(metadata, None, "Veo un coche").determinacy)
        self.assertEqual(Definiteness.indef_nonspec, ObjectSentence(metadata, None, "Veo a un coche").determinacy)

In [11]:
# Run tests
test = MyTest()
test.test_determine_def()

/Veo a mis hijos / a ADP
/Veo a la televisión / a ADP
/Veo a un coche / a ADP


In [12]:
class Sentence:
    def __init__(self, year, title, author, original_text, modern_text, nlp):
        '''
        year: the year in which the sentence was written
        text_title: title of the text
        original_text: the complete original text
        modern_text: the modern text with fixed spelling
        tagged_text: the nlp tagged text
        '''
        self.metadata = MetaData(year, title, author)
        self.original_text = original_text
        self.modern_text = modern_text
        self.tagged_text = nlp(modern_text)
        
        self.set_has_object()
        self.set_has_object_with_a()
        
    
    def __cmp__(self, other):
        return cmp(self.metadata, other.metadata)
    
    def __repr__(self):
        return "%s (%s) [%s, %s, %s]" % (self.original_text, self.modern_text, self.metadata.text_title, self.metadata.author, self.metadata.year)
    
    def display_tagged_text(self):
        displacy.render(self.tagged_text, style='dep', jupyter = True, options = {'distance': 120})
    
    # ---------------------------------------------
    # Setters
    
    def set_has_object(self):
        for w in self.tagged_text:
            if w.dep_ == 'obj':
                self.has_object = True
                return
        self.has_object = False
    
    def find_obj_subsentences(self):
        '''
Text:  le
    Dependency:  obj
    POS:  PRON
    Head:  di
Text:  di
    Dependency:  conj
    POS:  VERB
    Head:   
Text:  a
    Dependency:  case
    POS:  ADP
    Head:  hermana
Text:  mi
    Dependency:  det
    POS:  DET
    Head:  hermana
Text:  hermana
    Dependency:  obj
    POS:  NOUN
    Head:  di
        '''
        obj_subsentences = []
        verbs = [] # to check we don't collect the same verb with multiple objects multiple times
        for w in self.tagged_text:
            if w.dep_ == 'obj':
                for v in self.tagged_text:
                    if str(w.head) == v.text and v.text not in verbs:
                        verbs.append(v.text)
                        sen = ObjectSentence(self.metadata, v.subtree, self)
                        obj_subsentences.append(sen)
        return obj_subsentences
    
    def set_has_object_with_a(self):
        for w in self.tagged_text:
            if w.dep_ == 'obj':
                for v in self.tagged_text:
                    #print("Text ", v.text)
                    if str(v.head) == w.text:
                        self.has_object_with_a = (v.text == 'a' or v.text == 'al')
                        return
        self.has_object_with_a = False
    
    def set_info(self):
        '''
        TODO TODO TODO See stappenplan below
        '''
        if not self.has_object:
            self.verb = None
            self.determinacy = None
            self.animacy = None
            self.position_object = None
        else:
            return # See stappenplan below
    
    
    
    # --------------------------------------------
    # Getters
    
    def get_has_object():
        return self.has_object()


class MyTest(unittest.TestCase):
    def test_has_object_with_a(self):
        '''
        Tests the has_object_with_a function with different examples
        Also used to see if something breaks after adjusting
        '''
    
        #one syllable word, with bridge
        self.assertEqual([True], build_syllable_representation('ˈbu̠t͡s'))
        

In [13]:
from xml.dom import minidom

import lxml.etree as etree
from spacy import displacy

In [14]:
import es_core_news_sm
nlp = es_core_news_sm.load()

In [15]:
def print_info(doc):
    for w in doc:
        print("Text: ", w.text)
        print("    Dependency: ",w.dep_)
        print("    POS: ",w.pos_)
        print("    Head: ",w.head)
        

In [16]:
def fix_sentence(sentence):
    '''
    Parser reads things like "v [ uest ] ros"
    We try to fix that to help with the POSing later
    
    sentence: sentence to be fixed
    return: fixed sentence
    '''
    s = sentence.replace(" [ ] ", "")
    s = s.replace(' [ ', '')
    s = s.replace(' ] ', '')
    s = s.replace('havernos', 'habernos')
    s = s.replace('vn', 'un')
    s = s.replace('auia', 'habia')
    s = s.replace('aver', 'haber')
    return s
    

def parse_file(filename, nlp):
    f = minidom.parse(filename)
    
    sentence_ending_chars = ["."]
    
    root = f.getElementsByTagName('TEI')[0]
    bibl = root.getElementsByTagName('teiHeader')[0].getElementsByTagName('fileDesc')[0].getElementsByTagName('sourceDesc')[0].getElementsByTagName('bibl')[0]
    year = bibl.getElementsByTagName('date')[0].firstChild.data
    title = bibl.getElementsByTagName('title')[0].firstChild.data
    author = bibl.getElementsByTagName('author')[0].firstChild.data
    
    session = root.getElementsByTagName('text')[0].getElementsByTagName('body')[0].getElementsByTagName('div')[0].getElementsByTagName('ab')[0]
    sentence_original = ""
    sentence_modern = ""
    sentences = []
    #print(session.toprettyxml())
    for word in session.getElementsByTagName('*'):
        if word.tagName in ['pc', 'w', 'c']:
            #print(word.toprettyxml())
            if len(word.getElementsByTagName('choice'))<=0:
                orig = word.firstChild.data
                reg = orig
            else: 
                choice = word.getElementsByTagName('choice')[0]
                orig = choice.getElementsByTagName('orig')[0].firstChild.data
                reg = choice.getElementsByTagName('reg')[0].firstChild.data
            if word.tagName == "pc" and orig != ",": # in sentence_ending_chars:
                sentence_original += orig
                sentence_modern += reg
                sentence_modern = fix_sentence(sentence_modern)
                sentences.append(Sentence(year, title, author, sentence_original, sentence_modern, nlp))
                sentence_original = ""
                sentence_modern = ""
            elif not reg.isupper() and reg != 'A': #we do not do upper case ones because those are names in plays
                sentence_original += " " + orig
                sentence_modern += " " + reg
    return sentences

#print(parse_file("../BVC/10107.xml", nlp))


In [17]:
test_file = parse_file("../BVC/{}.xml".format(nr_file_to_parse), nlp)

In [18]:
print(test_file[34])
test_file[34].display_tagged_text()
print(test_file[34].has_object)
print(len(test_file))

 Mal pecado! ( Mal pecado!) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]


False
657


In [19]:
def collect_object_sentences(sentences):
    '''
    sentences: list of sentences (so list of Sentence type objects)
    
    returns: list of sentences that contain an object
    '''
    object_sentences = []
    object_sentences_a = []
    for sentence in sentences:
        if sentence.has_object:
            object_sentences.append(sentence)
            
        if sentence.has_object_with_a:
            object_sentences_a.append(sentence)
    return object_sentences, object_sentences_a

In [20]:
# Interesting practice sentence: Muélanle a palos al secretario

obj_sen, obj_sen_a = collect_object_sentences(test_file)
print(len(obj_sen), len(obj_sen_a))
#print(obj_sen_a[34])
#print_info(obj_sen_a[34].tagged_text)
#obj_sen_a[34].display_tagged_text()

497 44


In [21]:
for w in obj_sen_a[1].tagged_text:
    for s in w.subtree:
        print(s)
    print("-------------------")

 
por
qué
no
le
tentó
a
él
,
sino
por
verle
leal
y
constante
?
,
-------------------
por
-------------------
por
qué
-------------------
no
-------------------
le
-------------------
por
qué
no
le
tentó
a
él
,
sino
por
verle
leal
y
constante
-------------------
a
-------------------
a
él
-------------------
,
-------------------
sino
-------------------
por
-------------------
,
sino
por
verle
leal
y
constante
-------------------
leal
y
constante
-------------------
y
-------------------
y
constante
-------------------
?
-------------------
,
-------------------


In [22]:
for s in obj_sen_a:
    print(s)

 qué se espera de quien tuvo al diablo por maestro? ( qué se espera de quien tuvo al diablo por maestro?) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 por qué no le tentó a él , sino por verle leal y constante?, ( por qué no le tentó a él , sino por verle leal y constante?,) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 y no viéndose bastante para tentallo y vendello , dióle a ella el cargo de ello como a quien le va adelante en engaño; ( y no viendose bastante para tentarlo y vendello , dióle a ella el cargo de ello como a quien le va adelante en engaño;) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 Quán ageno estáys en esso , Fileno , de lo que devéys sentir , si pensáys ser maldezir llamar al negro moreno. ( cuán ajeno estáis en esso , Fileno , de lo que debéis sentir , si pensáis ser maldecir llamar al negro moreno.) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 Solamente hablamos aquí al presente de los males que la hembra en el mundo causa y siembra 

In [23]:
for s in obj_sen:
    print(s)

 Bien se conoçe , Fileno , que andáys alegre y ufano. ( Bien se conoce , Fileno , que andáys alegre y ufano.) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 No os pareçe , Alethio hermano , que es bien gozar de lo bueno y alaballo? ( No os parece , Alethio hermano , que es bien gozar de lo bueno y alaballo?) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 Quanto más yo , que me hallo preso de lindos amores y tan rico de favores , que peno quando los callo. ( Quanto más yo , que me hallo preso de lindos amores y tan rico de favores , que peno quando los callo.) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 Sinrazón les hazéys , si tales son , pues la ley de amor perfeto nos manda tener secreto lo que está en el coraçón. ( Sinrazón les hacéis , si tales son , pues la ley de amor perfecto nos manda tener secreto lo que está en el coraçón.) [Dialogo de mujeres, Cristóbal de Castillejo, 1544]
 Bien sería , pero yo no tomaría plazer grande ny senzillo a troque de no dezillo

In [24]:
obj_sub_sentences = []
only_pron_counter = 0
only_adj_counter = 0
for sen in obj_sen:
    sub_sentences_sen = sen.find_obj_subsentences()
    for sub in sub_sentences_sen:
        #if not sub.has_only_pron and not sub.has_only_adj:
        if not sub.has_only_adj:
            obj_sub_sentences.append(sub)
        elif sub.has_only_adj:
            only_adj_counter += 1
        #else:
        #    only_pron_counter += 1

for obj_sub_sent in obj_sub_sentences:
    print(obj_sub_sent, obj_sub_sent.verb)

print(len(obj_sub_sentences))
#only_pron_obj_sent = [sent for sent in obj_sub_sentences if sent.has_only_pron]
print("Removed {} sentences with only adjective objects".format(only_adj_counter))
print("Removed {} sentences with only pronouns".format(only_pron_counter))

/  No os parece , Alethio hermano , que es bien gozar de lo bueno y alaballo ? / bueno ADJ
/gozar de lo bueno y alaballo / bueno ADJ
/, que me hallo preso de lindos amores y tan rico de favores , que peno quando los callo / preso ADJ
/, que me hallo preso de lindos amores y tan rico de favores , que peno quando los callo / lindos NOUN
/, que me hallo preso de lindos amores y tan rico de favores , que peno quando los callo / rico ADJ
/, que me hallo preso de lindos amores y tan rico de favores , que peno quando los callo / peno ADJ
/, pero yo no tomaría placer grande ny sencillo a troque de no decirlo y gozar en conpañía mi favor / grande ADJ
/, pero yo no tomaría placer grande ny sencillo a troque de no decirlo y gozar en conpañía mi favor / sencillo VERB
/, pero yo no tomaría placer grande ny sencillo a troque de no decirlo y gozar en conpañía mi favor / a ADP
/quán poco duró / poco ADV
/de donde para burlar y mentir quedó vezada / vezada ADJ
/y cobró súbitamente mal siniestro para ma

/que honrada , muger de pompa y arreo , adoleçió de desseo de una saya verdugada muy lozana , a su parecer galana , que yendo a la iglesia vio , de que luego le tomó ynfinitíssima gana ; / lozana ADJ
/que honrada , muger de pompa y arreo , adoleçió de desseo de una saya verdugada muy lozana , a su parecer galana , que yendo a la iglesia vio , de que luego le tomó ynfinitíssima gana ; / a ADP
/, de que luego le tomó ynfinitíssima gana ; / ynfinitíssima ADV
/y tornada a casa muy congoxada / a ADP
/y tornada a casa muy congoxada / congoxada ADJ
/a comer començósse a entristeçer y mostrar muy fatigada / començósse ADJ
/a comer començósse a entristeçer y mostrar muy fatigada / a ADP
/a comer començósse a entristeçer y mostrar muy fatigada / fatigada ADJ
/y mostrar muy fatigada / fatigada ADJ
/, y como que enferma estaba , la causa disimulaba de la pasión que tenía / de ADP
/  " no tengáis miedo que de este mal muera ya vuestra muger , o no habrá mercaderes en Toledo . / muera NOUN
/darle al

/  Quál es ésa ? /   SPACE
/que , aunque profesa , tomaría por partido estar más so su marido que encima de su abadesa / estar AUX
/que , aunque profesa , tomaría por partido estar más so su marido que encima de su abadesa / más ADV
/  No conficsso tal pecado y vos me lo levantáys . / tal DET
/  No conficsso tal pecado y vos me lo levantáys . / vos ADJ
/por qué decís mal de las pobretas a la religión / de ADP
/lo que sé de esta cuenta , en que habrá más de cuarenta discretas , nobles , hermosas y aun algunas generosas que pudieran sin afrenta ser señoras / más ADV
/lo que sé de esta cuenta , en que habrá más de cuarenta discretas , nobles , hermosas y aun algunas generosas que pudieran sin afrenta ser señoras / de ADP
/lo que sé de esta cuenta , en que habrá más de cuarenta discretas , nobles , hermosas y aun algunas generosas que pudieran sin afrenta ser señoras / cuarenta NUM
/lo que sé de esta cuenta , en que habrá más de cuarenta discretas , nobles , hermosas y aun algunas generosa

/arrastradas por el polvo / por ADP
/que veo / que SCONJ
/que no traigo suffiçiençia ni caudal de poder / caudal NOUN
/para darsela a placeres / a ADP
/que para darsela a placeres tienen gracias singulares / a ADP
/que para darsela a placeres tienen gracias singulares / singulares ADJ
/para darnos pesares / pesares ADJ
/, y para darnos pesares bastantíssimos poderes / pesares ADJ
/do se encierra más luenga y áspera guerra que fue aquélla por Elena / más ADV
/do se encierra más luenga y áspera guerra que fue aquélla por Elena / áspera ADJ
/do se encierra más luenga y áspera guerra que fue aquélla por Elena / guerra NOUN
/do se encierra más luenga y áspera guerra que fue aquélla por Elena / aquélla NOUN
/que a pobres y ricos arrancan los corazones / pobres ADJ
/  cualesquier generaçiones tienen suertes espeçiales que loar : / espeçiales ADJ
/y quien lo contradixere tendrá tan mal parecer como vos / como SCONJ
/, que a las obras de natura contradigamos los dos locamente ; / locamente ADV


/que se regosta a la espiga candeal / a ADP
/no ay bolsa tan liberal / liberal ADJ
/que después que se regosta a la espiga candeal , no ay bolsa tan liberal que no se les haga angosta / a ADP
/, que muchos saben de coro , / de ADP
/a que nos dan ocasiones , cuchilladas y ruidos , / cuchilladas ADJ
/desonrrados por querellas / por ADP
/, donde quedaron tendidos , / tendidos ADJ
/que más hombres ha tragado en poniente , en Ytalia mayormente , que es sepulcro de naciones / más ADV
/  No se excusan disensiones do quiera que ay mucha gente ; / mucha DET
/que ay mucha gente / mucha DET
/andan otras aparçeras cerca de éstas , que no son menos molestas y son sus colaterales / otras DET
/  algunos las llaman amas honestas , viejas pobretas , cuyo nombre es alcahuetas , sin más andar por las ramas . / honestas ADJ
/  algunos las llaman amas honestas , viejas pobretas , cuyo nombre es alcahuetas , sin más andar por las ramas . / pobretas ADJ
/, que de nadie tienen duelo por comer a costa ajena / 

/que no ay reparo , porque os venden siempre caro y conpran de vos barato qualquier cosa / barato ADJ
/, porque os venden siempre caro y conpran de vos barato qualquier cosa / caro ADJ
/, porque os venden siempre caro y conpran de vos barato qualquier cosa / barato ADJ
/y conpran de vos barato qualquier cosa / barato ADJ
/, que yo reçibiesse aquello y que pusiese sobre ello si alguna cosa faltaba , y tomados a cuenta los lacerados anillejos y manilla , / lacerados ADJ
/, que yo reçibiesse aquello y que pusiese sobre ello si alguna cosa faltaba , y tomados a cuenta los lacerados anillejos y manilla , / manilla NOUN
/, y tomados a cuenta los lacerados anillejos y manilla , / lacerados ADJ
/, y tomados a cuenta los lacerados anillejos y manilla , / manilla NOUN
/para beverlo de presto tras la lunbre / de ADP
/, con que se acuesta y levanta para darnos pesadumbre y cuidados , con reportes y recados las más vezes mentirosos , pero caros y costosos , enbueltos en mill enhados de dolor / cuid

/que os escriví , porque allí me hallaréis , y corred / escriví
/, porque allí me hallaréis , y corred / hallaréis
/que hurtava bogas / hurtava
/la burlaba , él al fin / burlaba
/, y que la burlaba , él al fin quedó burlado , porque yendo a su posada corriendo , a un amigo lo mostró , y abierto el cofre / burlaba
/, porque yendo a su posada corriendo , / yendo
/a un amigo lo mostró , y abierto el cofre / mostró
/, y abierto el cofre / abierto
/que lleváys hurtado el tesoro que aquí va , guardaldo , que no os será por mí jamás demandado ni pedido / lleváys
/, que no os será por mí jamás demandado ni pedido / demandado
/que me queda / queda
/  " Él quedó corrido quando se vio hecho por fuerça ser padre del ynfante , cuya madre nunca jamás conoçió . / quedó
/hecho por fuerça / vio
/, pues se están por sí loadas aunque vos las desloéys / están
/si tenéis prestas las manos del mal dezir para llagar y herir también las viudas honestas / prestas
/y herir también las viudas honestas / herir
/l

/condenarlas todas / perfectas
/salen cuentos muy reales y algunos malos recados y partos disimulados escondidos en costales por rincones , con sutiles ynventiones de dar color a lo hecho , porque no pierdan derecho sus honras y presunciones / salen
/de dar color a lo hecho / dar
/, porque no pierdan derecho sus honras y presunciones / pierdan
/que en demandas y respuestas se saben bien gobernar / saben
/  Mas aun éstas que en demandas y respuestas se saben bien gobernar , se podrían perdonar , porque ay otras deshonestas , demandadas y de esto tan descuidadas con el vicio a que se dan , que por do quiera que van dejan rastros y pisadas del delito , que llega a ser ynfinito desque una vez se comienza , o teniendo en él vergüenza ni modo en el apetito . / saben
/, porque ay otras deshonestas , demandadas y de esto tan descuidadas con el vicio a que se dan , que por do quiera que van dejan rastros y pisadas del delito , que llega a ser ynfinito desque una vez se comienza , o teniendo en 

/, por faltar la libertad y lugar que sobran a las solteras , con gracias de mill maneras de que suelen se hallar rodeadas , y muchas de ellas dotadas de virtudes exçelentes , no pocas de las presentes y muchas de las passadas , sus iguales , Thays , Flora y otras tales / faltar
/que sobran a las solteras , con gracias de mill maneras de que suelen se hallar rodeadas , / sobran
/se hallar rodeadas / hallar
/, que sabía las siete artes liberales / sabía
/las llamo los cantares del reclamo o cantos de las serenas mal sentidos / llamo
/que los dan / dan
/tras que se van / van
/al que la prueva al precio de la de Adán / prueva
/, que conviene hecharles sueltas porque las han menester / conviene
/porque las han menester / menester
/a la qual la libertad les da grandes ocasiones / da
/a que después se van dando , por oficio y ley tomando la vida desvergonzada , que es la fuente de do sale la corriente de tanta vellaquería , teniendo por granjería vendernos públicamente sus deleites , usando 

In [25]:
# Put all sentences into an csv file
def save_obj_sentences(filename, sentences):
    '''
    
    filename: name of file to be saved to
    sentences: list of ObjectSentences to be saved
    return: None
    '''
    file = open(filename, 'w', newline ='')
 
    with file:
        # identifying header 
        header = ['year', 'title', 'author', 'orig_sentence', 'modern_sentence', 
                  'obj_sentence', 'has_a', 'verb', 'definiteness', 'definiteness_cor', 
                  'order', 'animacy', 'corrected_order', 'remove', 'has_dative', 'other_change']
        writer = csv.DictWriter(file, fieldnames = header, delimiter='\t')
        writer.writeheader()
        for sentence in sentences: 
            writer.writerow({
                'year' : sentence.metadata.year,
                'title' : sentence.metadata.text_title,
                'author' : sentence.metadata.author,
                'orig_sentence' : sentence.original_text,
                'modern_sentence' : sentence.modern_text,
                'obj_sentence' : str(sentence),
                'has_a' : sentence.has_a,
                'verb' : sentence.verb,
                'definiteness' : sentence.determinacy,
                'definiteness_cor' : "",
                'animacy' : "",
                'order' : sentence.order
            })
    
    

In [26]:
save_obj_sentences("parsed/{}.tsv".format(nr_file_to_parse), obj_sub_sentences)

# Things that would be nice to add:
- source year besides just year in which text was written
- further data cleaning
- match xml annotation with pos tagger annotation
- ~"si anda suelto" now added cause "suelto" recognized as adjective object. Filter those out~
- Lemmatization of the verbs
- **Filter out dative sentences?**

Stappenplan
- Googlen of er code is die alleen substukjes van zinnen pakt
- Als niet zelf recursieve functie maken die alleen ww (wat obj heeft) en alles wat eronder hangt pakt. Mss alleen totdat er een nieuw werkwoord bijkomt? 
- Daarmee dan bepalen waar ww staat en waar obj staat (links of rechts van ww)
- Zelfde manier code maken die alleen object en alles wat daaronder hangt pakt
- Daarmee bepalen wat de determinancy is (en evt animacy)



Has_object_subsentence class maken, met attributes:
- metadata?
- has_a
- only pronoun objects?
- verb
- rel position verb and object
- animacy
- determinancy

New metadata object maken?